###  OPTICS algorithm implementation on single machine

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math

In [2]:
def distance(point1,point2):
    """
    cal distance between two points.
    
    """
    return np.linalg.norm(point1-point2)

In [3]:
def fit_ball_tree(x):
    """
    Ball Tree Algorithm
    """
    if x.shape[0]==1:
        node=dict({'pivot':x[0],
                   'son1':None,
                   'son2':None,
                   'radius':0
                   })
        return node
    c=np.argmax(np.std(x,axis=0)) #c is the "widest" dimension
    p1=x[np.argmin(x[:,c])]
    p2=x[np.argmax(x[:,c])]
    p=(p1+p2)/2 #p is the center of of dimension c
    radius=max(np.linalg.norm(x-p,axis=1)) #maximum distance from c to each point (radius)
    x1,x2=[],[]
    
    # divide x into two subset according to the distance to p
    for i in range(x.shape[0]):
        if np.linalg.norm(x[i]-p1)<np.linalg.norm(x[i]-p2):
            x1.append(x[i])
        else:
            x2.append(x[i])

    x1=np.array(x1)
    x2=np.array(x2)

    # build left sub-tree and right-sub tree recursively
    son1=fit_ball_tree(x1)
    son2=fit_ball_tree(x2)
    node=dict({'pivot':p,
               'son1':son1,
               'son2':son2,
               'radius':radius
                })
    return node


    
def ball_tree_search(k,t,node):
    """
    Search k neighbor using ball tree
    """
    global Q,q
    # Q is k-neighbor list, q is k-neighbor distance
    
    if np.linalg.norm(t-node['pivot'])-node['radius']>=np.max(q):
        return
    if node['son1']==None and node['son2']==None:
        if Q.shape[0]==k:
            Q=np.delete(Q,np.argmax(q),axis=0)
            q=np.delete(q,np.argmax(q))
        Q=np.append(Q,[node['pivot']],axis=0)
        q=np.append(q,np.linalg.norm(t-node['pivot']))
    else:
        ball_tree_search(k,t,node['son1'])
        ball_tree_search(k,t,node['son2'])

In [4]:
def CorePoints(Eps,MinPts,data,root):
    """
    Search CorePoints of given data using ball-tree
    """
    core_Points = {}
    core_Distance = {}
    
    for current_point in range(len(data)):
        neighbor = []
        dis = []
        
        global Q,q
        
        Q=np.array([[np.inf,np.inf]]) #global var
        q=np.array([np.inf]) #global var
        
        
        ball_tree_search(k=MinPts,t=data[current_point],node=root)
        
        #every point is core point, thus neighbors = all points 
        core_Points[current_point] = np.arange(len(data))

        core_Distance[current_point] = np.max(q)
            
    
    return core_Points,core_Distance

In [5]:
def reachability_distance(data,core_Points,core_Distance):
    """
    Cal reachability distance 
    """
    rd_Matrix_shape = (data.shape[0],data.shape[0])
    rd_Matrix = np.random.rand(rd_Matrix_shape[0],rd_Matrix_shape[1])+float('inf')
    
    for current_point in range(len(data)):
        for other_point in core_Points.keys():
            rd_Matrix[current_point,other_point] = max(core_Distance[other_point],distance(data[current_point],data[other_point]))
    
    return rd_Matrix

In [6]:
def Update(neighbor,point):
    """
    Update Function
    """
    global seed,Orderedlist,core_Points,rd_Matrix
    
    for neihbor_point in neighbor:
        if neihbor_point not in Orderedlist:
            if neihbor_point not in seed.keys():
                seed[neihbor_point] = rd_Matrix[point][neihbor_point]
            else:         
                seed[neihbor_point] = min(rd_Matrix[point][neihbor_point],seed[neihbor_point])
                
    # seed = dict(sorted(seed.items(), key=lambda x: x[1], reverse=False))
    
    # while(len(seed)>0):
    #     current_point = list(seed)[0]
    #     dis = seed[current_point]
    #     seed.pop(current_point)
    #     if current_point not in Orderedlist:
    #         Orderedlist.append(current_point)
    #         Reach_distance.append(dis)
    #         if current_point in core_Points.keys():
    #             neighbor=core_Points[current_point]
    #             Update(seed,Orderedlist,neighbor,core_Points,current_point,rd_Matrix,Reach_distance)
            
    # return seed,Orderedlist,Reach_distance
    return seed

In [7]:
def Optics(Eps,MinPts,data):
    
    global seed,Orderedlist,core_Points,rd_Matrix
    
    Orderedlist = [] 
    
    Reach_distance = []

    root=fit_ball_tree(data)
     
    core_Points,core_Distance = CorePoints(Eps,MinPts,data,root)
    rd_Matrix = reachability_distance(data,core_Points,core_Distance)

    last_point = list(core_Points)[0]
    
    
    for point in core_Points.keys():
        if point not in Orderedlist: 
            
            neighbor = core_Points[point]
            Reach_distance.append(rd_Matrix[point][last_point])
            Orderedlist.append(point)
            
            seed = {}
              
            seed = Update(neighbor,point)
            
            for key in seed.keys():
                neighbor = core_Points[key]
                Orderedlist.append(key)
                seed = Update(neighbor,key)
                
    return Orderedlist,Reach_distance

In [8]:
def draw(Eps,MinPts,Orderedlist,Reach_distance,data):
    """
    A visualization method for two dimension data only
    """
    ind = [0]
    data_split = []
    Noise = []
    
    for i in range(len(Reach_distance)):
        if Reach_distance[i]>Eps:
            ind.append(i)
            
    if ind[-1]!=len(Reach_distance):
        ind.append(len(Reach_distance))
    
    for i in range(len(ind)-1):
        if ind[i+1]-ind[i] < MinPts:
            Noise.append(ind[i])
            ind[i]=ind[i]-1
            if ind[i+1] == len(Reach_distance):
                Noise.append(ind[i+1])
         #   ind.remove(ind[i])
    for i in Noise:
        if i in ind:
            ind.remove(i)
    
    
    for i in range(len(ind)-1):
        data_split.append(Orderedlist[ind[i]:ind[i+1]])
    
    data = pd.DataFrame(data)
    data.columns=['x','y']
    data['label'] = -1
    
    for x in range(len(data_split)):
        for y in data_split[x]:
            data.iloc[y,2]=x
            
    
    for i in Noise:
        data.iloc[i-1,2] = x+1
        
    fig = plt.figure(figsize=(12,8),dpi=200)
    colors = ['red','green','blue','orange','yellow','purple','pink','black']
    plt.scatter(data['x'], data['y'], c=data['label'], cmap=matplotlib.colors.ListedColormap(colors))
    return data

In [12]:
data = np.loadtxt('Data/a1.txt')

In [13]:
ord,res = Optics(float('inf'),5,data)

In [ ]:
# draw(3,50,ord,res,data)